In [2]:
import pandas as pd

df = pd.read_json("combined_dataset.json", lines=True)
df.to_csv("Dataset.csv", index=False)

In [3]:
df.isnull().sum()
df.drop_duplicates(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2752 entries, 0 to 3511
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Context   2752 non-null   object
 1   Response  2752 non-null   object
dtypes: object(2)
memory usage: 64.5+ KB


In [5]:
import re

def ProcessAndCombineText(text1,text2):
    # Remove leading and trailing whitespace
    text1.strip()
    text2.strip()

    # Replace newline characters with spaces
    text1 = text1.replace("\n", " ")
    text2 = text2.replace("\n", " ")

    # Remove multiple spaces
    text1 = re.sub(r'\s+', ' ', text1)
    text2 = re.sub(r'\s+', ' ', text2)

    # Add space after punctuation if not present
    text1 = re.sub(r'([\.\,\?\!])(\w)', r'\1 \2', text1)
    text2 = re.sub(r'([\.\,\?\!])(\w)', r'\1 \2', text2)

    # Add label to the sentences for better understanding
    text1 = "Client: " + text1
    text2 = "Assistant: " + text2

    # Combine the two texts
    text = text1 + "\n" + text2

    return text

df['combined'] = df.apply(lambda x: ProcessAndCombineText(x['Context'], x['Response']), axis=1)

In [7]:
print(df['combined'].iloc[1])

Client: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?
Assistant: Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone. Changing our feelings is like changing our thoughts - it's hard to do. Our minds are so amazing that the minute you change your thought another one can be right there to take it's place. Without your permission, another thought can just pop in there. The new thought may feel worse than the last one! My guess is that you have tried several things to improve this on your own even before reaching out on here. People often try t

In [ ]:
from openai import OpenAI

client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['embedding'] = df['combined'].apply(lambda x: get_embedding(x))


# df.to_csv('embedded_1k_reviews.csv', index=False)

In [ ]:
import numpy as np

# Convert the embeddings to a numpy array
embedding_matrix = np.array(df['embedding'].to_list()).astype(np.float32)

In [ ]:
import faiss

# Get the dimension of the embeddings
dimension = embedding_matrix.shape[1]

# Create a FAISS index that uses L2 distance 
index = faiss.IndexFlatL2(dimension) 

# Add the embeddings to the index
index.add(embedding_matrix)

In [ ]:
# Save the index to a file so we can use it later
faiss.write_index(index, "sanity_llm_index.faiss")

In [ ]:
# Read the index from the file
faiss.read_index("sanity_llm_index.faiss")

query = "Testing"

# Embed the query and then turnining it into a numpy array with float32 type
def prepare_query_embedding(query):
    return np.array([get_embedding(query)]).astype(np.float32)